<a href="https://colab.research.google.com/github/MaQuest/Summer2021/blob/main/Charity_reco_sys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install sentence-transformers

     |████████████████████████████████| 92kB 11.9MB/s 
     |████████████████████████████████| 2.5MB 21.1MB/s 
     |████████████████████████████████| 1.2MB 43.0MB/s 
     |████████████████████████████████| 3.3MB 34.0MB/s 
     |████████████████████████████████| 901kB 42.8MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.0.0-cp37-none-any.whl size=126711 sha256=2f52aa46dfbf693065a965d5f08579f0b73d2c8137fddb10e32b55d2325d1525
  Stored in directory: /root/.cache/pip/wheels/38/d2/98/d191289a877a34c68aa67e05179521e060f96394a3e9336be6
Successfully built sentence-transformers
ERROR: transformers 4.8.2 has requirement huggingface-hub==0.0.12, but you'll have huggingface-hub 0.0.13 which is incompatible.


In [5]:
import pandas as pd 
import numpy as np
import json
import os
from sentence_transformers import SentenceTransformer, util
import pickle

In [7]:
ratings = pd.read_csv('/content/drive/MyDrive/Collaborative Filtering Dataset/dataset/ratings.csv')
charities_AB = pd.read_csv('/content/drive/MyDrive/Collaborative Filtering Dataset/dataset/charity_ab.csv',index_col=0)


ratings_new = charities_AB.join(ratings).drop(['timestamp'],axis = 1)
ratings_new.at[2781,'rating']= 4
ratings_new.at[3379,'rating']= 5
ratings_new.at[8033,'rating']= 5

user_ratings = ratings_new.pivot_table(index = ['userId'], columns = ['Charity Name'] , values = 'rating')
user_ratings = user_ratings.fillna(0)
item_similiarity_df = user_ratings.corr(method = 'pearson') 


def get_similiar_movies(movie_name,user_rating):
    similiar_score = item_similiarity_df[movie_name]*(user_rating-2.5)
    similiar_score = similiar_score.sort_values(ascending = False)

    return similiar_score



df = pd.read_csv('/content/drive/MyDrive/Collaborative Filtering Dataset/dataset/charity_ab_updated.csv')

papers = df['Charity Name'].values+" "+df['Charity Category'].values

print(len(papers), "Charities")


9308 Charities


In [8]:
file = open('/content/drive/MyDrive/Collaborative Filtering Dataset/dataset/finalized_charity_ab.sav', 'rb')

# dump information to that file
model = pickle.load(file)

# close the file
file.close()

#To encode the descriptions to a single string
paper_texts = [paper for paper in papers]

corpus_embeddings = model.encode(paper_texts, convert_to_tensor=True)


In [16]:
from termcolor import colored

charities = []
def search_papers(title):
   query_embedding = model.encode(title+'[SEP]', convert_to_tensor=True) # Converts to tensor
   search_hits = util.semantic_search(query_embedding, corpus_embeddings)
   search_hits = search_hits[0]
   count = 0
   top_header = "Top 10 charities related"
   print(colored(top_header,'blue'))
   for hit in search_hits:
         related_paper = papers[hit['corpus_id']]
         count += 1
         print(str(count)+ ")" + " Description of charity : " +related_paper)
         print("similiarity score of " + str(format(search_hits[count-1]['score'],".2f")))
         subsetDataFrame = df[df['Charity Name']+" "+df['Charity Category'] == related_paper]
         k=subsetDataFrame.values
         a_string = "Name of Organization"
         print("\033[1m" + a_string + "\033[0m")
         print(colored(str(k[0][1]), 'red'))
         charities.append(str(k[0][1]))
         
# search_papers(title=input(str()))

search_papers("lungs")

Top 10 charities related
1) Description of charity : THE ALBERTA LUNG FOUNDATION Foundations
similiarity score of 0.83
Name of Organization
THE ALBERTA LUNG FOUNDATION
2) Description of charity : ALBERTA LUNG ASSOCIATION Core Health Care
similiarity score of 0.81
Name of Organization
ALBERTA LUNG ASSOCIATION
3) Description of charity : Al and Janet Petersen Foundation Foundations
similiarity score of 0.81
Name of Organization
Al and Janet Petersen Foundation
4) Description of charity : Dear Heart Foundation Foundations
similiarity score of 0.80
Name of Organization
Dear Heart Foundation
5) Description of charity : STANDARD COMMUNITY CHEST Foundations
similiarity score of 0.80
Name of Organization
STANDARD COMMUNITY CHEST
6) Description of charity : Bruce Hagstrom Foundation Foundations
similiarity score of 0.80
Name of Organization
Bruce Hagstrom Foundation
7) Description of charity : Summit Foundation For Cystic Fibrosis Foundations
similiarity score of 0.80
Name of Organization
Summi

In [18]:
# CORPUS CHRISTI MALE CHORALE SOCIETY

question = str(input("would you like to give a ratings to any of the top 10 recommendations? : "))
whichone = ""
if (question == "yes") or (question == " Yes"):
    whichone = str(input("Please enter the name of the charity : "))
    ratingof = int(input("Please Enter the rating you would like to give : "))
    print(ratingof)
    print(whichone)
    print(type(ratingof))

action_lover = [(whichone,ratingof)]

similiar_movies = pd.DataFrame()

for movie,rating in action_lover:
    similiar_movies = similiar_movies.append(get_similiar_movies(movie,rating),ignore_index= True)

similiar_movies.head()
similiar_movies.sum().sort_values(ascending = False)[:10]


# Cosmopolitan Music society 
# Cornerstone youth
# CORONATION & DISTRICT HISTORICAL & MUSEUM SOCIETY
# COVENANT BAY BIBLE CAMP SOCIETY

would you like to give a ratings to any of the top 10 recommendations? : yes
Please enter the name of the charity : CORPUS CHRISTI MALE CHORALE SOCIETY
Please Enter the rating you would like to give : 5
5
CORPUS CHRISTI MALE CHORALE SOCIETY
<class 'int'>


DELBURNE UNITED CHURCH                               2.5
DAVID THOMPSON BIBLE CAMP                            2.5
COVENANT BAY BIBLE CAMP SOCIETY                      2.5
COSMOS COMMUNITY DISABILITY SERVICES FOUNDATION      2.5
COSMOPOLITAN MUSIC SOCIETY                           2.5
CORONATION & DISTRICT HISTORICAL & MUSEUM SOCIETY    2.5
CORNERSTONE YOUTH CENTRE (CALGARY) SOCIETY           2.5
CORNERSTONE CHRISTIAN SCHOOL SOCIETY                 2.5
COOKIE JAR NURSERY ASSOCIATION                       2.5
CONSORT PASTORAL CHARGE                              2.5
dtype: float64

In [59]:
ratings_local = [5,2,3,5,3,4,1,3,3,1]
dat = {'Charity Name':charities,'rating':ratings_local}
df_local = pd.DataFrame(dat)

df_local.to_csv('charity_ab_updated_3.csv')
!cp charity_ab_updated_3.csv "drive/My Drive/"

In [ ]:
df1 = pd.read_csv('/content/charity_ab_local.csv',index_col=0)
df2 = pd.read_csv('/content/charity_ab_updated_2.csv',index_col=0)
df3 = pd.read_csv('/content/charity_ab_updated_3.csv',index_col=0)

frames = [df1, df2, df3]

result = pd.concat(frames)

result

In [ ]:
user_ratings_char = result.pivot_table(columns = ['Charity Name'] , values = 'rating')

user_ratings_char

In [83]:

user_ratings_char = user_ratings_char.fillna(0)

user_ratings_char.head()

Charity Name,ALBERTA LUNG ASSOCIATION,Al and Janet Petersen Foundation,BETHEL GOSPEL CHAPEL OF EDMONTON,BONNYVILLE VICTIM SERVICES SOCIETY,Beaverlodge & Grande Prairie Evangelical Strangers Assembly,Branch Out Neurological Foundation,Bruce Hagstrom Foundation,CARDSTON VICTIM SERVICES ASSOCIATION,"CORONATION PASTORAL CHARGE,",CORPUS CHRISTI MALE CHORALE SOCIETY,Dear Heart Foundation,EYEOPENER LIONS FOUNDATION (ELF),FORT EDMONTON FOUNDATION,INNISFAIL PASTORAL CHARGE,LETHBRIDGE CRIME STOPPERS ASSOCIATION,Mt 28 FOUNDATION,Operation Eyesight Canada,Optometry Giving Sight,"PAROISSE DE L'ASSOMPTION,",PAROISSE SAINTE ANNE,PAROISSE SAINTE-ANNE,SIMON HOUSE RESIDENCE SOCIETY,STANDARD COMMUNITY CHEST,Smiles from Spencer Foundation,Southeastern Alberta Sexual Assault Response Committee,Summit Foundation For Cystic Fibrosis,THE ALBERTA LUNG FOUNDATION,THE GIMBEL EYE FOUNDATION,VIEWPOINT FOUNDATION
rating,2.0,3.0,5.0,3.0,4.0,5.0,4.0,1.0,2.0,1.0,4.5,5.0,4.0,3.0,1.0,3.0,4.0,5.0,5.0,1.0,2.0,3.0,3.0,5.0,2.0,1.0,5.0,5.0,5.0


In [ ]:
item_similiarity_df_n = user_ratings_char.corr(method = 'pearson') 


item_similiarity_df_n.fillna(0)



In [86]:
def get_similiar_movies_2(movie_name,user_rating):
    similiar_score = item_similiarity_df_n[movie_name]*(user_rating)
    similiar_score = similiar_score.sort_values(ascending = False)

    return similiar_score

In [87]:
question = str(input("would you like to give a ratings to any of the top 10 recommendations? : "))
whichone = ""
if (question == "yes") or (question == " Yes"):
    whichone = str(input("Please enter the name of the charity : "))
    ratingof = int(input("Please Enter the rating you would like to give : "))
    print(ratingof)
    print(whichone)
    print(type(ratingof))

action_lover_n = [(whichone,ratingof)]

similiar_movies_n = pd.DataFrame()

for movie,rating in action_lover_n:
    similiar_movies_n = similiar_movies_n.append(get_similiar_movies_2(movie,rating),ignore_index= True)

similiar_movies_n.head()
similiar_movies_n.sum().sort_values(ascending = False)[:10]

would you like to give a ratings to any of the top 10 recommendations? : yes
Please enter the name of the charity : EYEOPENER LIONS FOUNDATION (ELF)
Please Enter the rating you would like to give : 5
5
EYEOPENER LIONS FOUNDATION (ELF)
<class 'int'>


VIEWPOINT FOUNDATION                                           0.0
INNISFAIL PASTORAL CHARGE                                      0.0
Al and Janet Petersen Foundation                               0.0
BETHEL GOSPEL CHAPEL OF EDMONTON                               0.0
BONNYVILLE VICTIM SERVICES SOCIETY                             0.0
Beaverlodge & Grande Prairie Evangelical Strangers Assembly    0.0
Branch Out Neurological Foundation                             0.0
Bruce Hagstrom Foundation                                      0.0
CARDSTON VICTIM SERVICES ASSOCIATION                           0.0
CORONATION PASTORAL CHARGE,                                    0.0
dtype: float64

In [47]:
df_local.to_csv('charity_ab_local.csv')
!cp charity_ab_local.csv "drive/My Drive/"